## Поиск похожих по эмбеддингам

Скачиваем датасет ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [24]:
# Импорт библиотек
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText

1. Объединить в одну выборку

In [25]:
# Загрузка данных

df1 = pd.read_csv('positive.csv', sep=';', header = None, usecols = [3], names=['tweet'])
df2 = pd.read_csv('negative.csv', sep=';', header = None, usecols = [3], names=['tweet'])

In [26]:
# Объединение двух датафреймов

df = df1.append(df2, ignore_index = True, sort = False)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tweet   226834 non-null  object
dtypes: object(1)
memory usage: 1.7+ MB


In [28]:
df = df.sample(100000)

In [29]:
df.sample(5)

,tweet
193052,"@RuFoteev Кстати, у нас похолодало, но без сне..."
8390,тебя отпускаю) а вот сердце держит. Выход один...
157404,http://t.co/oWupOY46Vy Го ребят. Скука одолева...
43250,Пора брать кредит на кассеты для бритья! :D ht...
165665,"RT @daskasmirnova: @mathsolver3 нет,ну просто ..."


2. На основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу.

In [30]:
exclude = string.punctuation
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [31]:
sw = set(get_stop_words("ru"))
sw

{'а',
 'алло',
 'без',
 'близко',
 'более',
 'больше',
 'будем',
 'будет',
 'будете',
 'будешь',
 'будто',
 'буду',
 'будут',
 'будь',
 'бы',
 'бывает',
 'бывь',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'важная',
 'важное',
 'важные',
 'важный',
 'вам',
 'вами',
 'вас',
 'ваш',
 'ваша',
 'ваше',
 'ваши',
 'вверх',
 'вдали',
 'вдруг',
 'ведь',
 'везде',
 'весь',
 'вниз',
 'внизу',
 'во',
 'вокруг',
 'вон',
 'восемнадцатый',
 'восемнадцать',
 'восемь',
 'восьмой',
 'вот',
 'впрочем',
 'времени',
 'время',
 'все',
 'всегда',
 'всего',
 'всем',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всю',
 'всюду',
 'вся',
 'всё',
 'второй',
 'вы',
 'г',
 'где',
 'говорил',
 'говорит',
 'год',
 'года',
 'году',
 'да',
 'давно',
 'даже',
 'далеко',
 'дальше',
 'даром',
 'два',
 'двадцатый',
 'двадцать',
 'две',
 'двенадцатый',
 'двенадцать',
 'двух',
 'девятнадцатый',
 'девятнадцать',
 'девятый',
 'девять',
 'действительно',
 'дел',
 'день',
 'десятый',
 'десять',
 'для',
 'до',
 'довольно',
 'до

In [32]:
test_str = '@Katyaandreeva95  :-|| RTeee бывает разное,  но всегда цветы  и  @k_katyaaa окей :‑) во что это  - значит'

In [33]:
line = re.sub(r'RT[\w]*', '', test_str)
line

'@Katyaandreeva95  :-||  бывает разное,  но всегда цветы  и  @k_katyaaa окей :‑) во что это  - значит'

In [34]:
line = re.sub(r'@[\w]*', '', line)
line

'  :-||  бывает разное,  но всегда цветы  и   окей :‑) во что это  - значит'

In [35]:
spls = ''.join(let for let in line.strip() if let not in exclude).split()
spls

['бывает',
 'разное',
 'но',
 'всегда',
 'цветы',
 'и',
 'окей',
 '‑',
 'во',
 'что',
 'это',
 'значит']

In [36]:
w2 = [word for word in spls if len(word)>2]
w2

['бывает', 'разное', 'всегда', 'цветы', 'окей', 'что', 'это', 'значит']

In [37]:
len(w2)<5

False

In [38]:
morpher = MorphAnalyzer()

In [39]:
n25tweet= [morpher.parse(word.lower())[0].normal_form for word in w2]
n25tweet

['бывать', 'разный', 'всегда', 'цветок', 'окей', 'что', 'это', 'значит']

In [40]:
res = [w for w in n25tweet if w not in sw and w != ""]
res

['бывать', 'разный', 'цветок', 'окей']

In [61]:
# Препроцессинг текста

def preprocess_txt(tweet, lim):
    line = re.sub(r'RT[\w]*', '', tweet)
    line = re.sub(r'@[\w]*', '', line)
    spls = "".join(let for let in line.strip() if let not in exclude).split()
    w2 = [word for word in spls if len(word)>2]
    if len(w2)<lim: return None
    else:
        n25tweet= [morpher.parse(word.lower())[0].normal_form for word in w2]
        res = [w for w in n25tweet if w not in sw and w != ""]
    return res

In [42]:
assert True

sentences = []

morpher = MorphAnalyzer()

for line in tqdm_notebook(df['tweet']):
    pre_tweet = preprocess_txt(line, 5)
    if pre_tweet is not None:
        sentences.append(pre_tweet) 

<ipython-input-42-7d4f07f17d7f>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(df['tweet']):


  0%|          | 0/100000 [00:00<?, ?it/s]

In [44]:
len(df)

100000

In [48]:
df.iloc[0:3]

,tweet
67374,"всего лишь потому,что не фотогеничны не будет ..."
155375,"Почти 13 лет тренировал(\nЖалко жалко,но такой..."
186076,"что-то счас денек такой,без солнца:(\nхочу ка..."


In [49]:
sentences[0:3]

[['потомучтый', 'фотогеничный', 'адекватный', 'жеэтый', 'справедливо'],
 ['тренировать', 'жалко', 'жалконо', 'специалист', 'работа', 'остаться'],
 ['чтоть', 'счас', 'денёк', 'такойбез', 'солнце', 'вчера']]

In [50]:
len(sentences)

86541

In [52]:
%%time
# Создание модели Word2Vec
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=3)

Wall time: 3.05 s


In [53]:
%%time
# Создание модели FastText
modelFT = FastText(sentences=sentences, vector_size=300, min_count=3, window=5, workers=8)

Wall time: 23.6 s


In [54]:
#Word2Vec?

In [57]:
%%time
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

counter = 0
index_map = {}

for line in tqdm_notebook(df['tweet']):
    n_w2v = 0
    n_ft = 0
    index_map[counter] = line
    tweet = preprocess_txt(line, 5)
    if tweet is not None:

        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in tweet:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
        counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

<timed exec>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/100000 [00:00<?, ?it/s]

Wall time: 5min 5s


True

In [69]:
def get_response(txt, index, model, index_map):
    pre_txt = preprocess_txt(txt, 0)
    vector = np.zeros(300)
    norm = 0
    if pre_txt is not None:
        for word in pre_txt:
            if word in model.wv:
                vector += model.wv[word]
                norm += 1
        if norm > 0:
            vector = vector / norm
        answers = index.get_nns_by_vector(vector, 5, )
        return [index_map[i] for i in answers]
    else:
        return []

3. Проверить насколько хорошо работают подходы

In [128]:
TEXT = "Продажа телефонов и аксессуаров с доставкой"

In [129]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Instagram   перегревает и разряжает в хлам мой телефон. Загрузка ЦП 46 ¤/¤ :( @instagram',
 'Я раб техники и интернета.Завис комп,ТУТ ЖЕ схватила телефон и включила инет,пока перезагружался комп.Вот так всегда.Либо ноут,либо телефон(',
 '@RomanWuduck Чени телефон) ну или в настройках инста полазь',
 '@bribuka161  что с тобой?((\nкуда пропала?\nчто со станицей в вк и телефоном?(((',
 'Я - доча, верни наконец мне мой телефон! Доча - ооой, ну Диля, ну щас! Еще чучуууть)))']

In [127]:
get_response(TEXT, ft_index, modelFT, index_map)

['Какая мелодия у тебя на телефоне? — только Rep))) http://t.co/LkDUQZLDHT',
 '@YanaYaroslavovn ой Янааа аахахахах у меня денег нет(',
 '@Lex3512 хахах даа надо :D \nтеперь вика это ларри а ларри вика :D',
 '@Winchesterovna я с телефона и того весь стол и телефон в кофе)))',
 '@kamilburanov @shokirjonov @BoburAkmalov Шуҳрат ака, мана уже СА ўқувчилари кўмакка келишди ))']

## Работает, но прямо скажем, так себе.